#### Imports and file names ####


In [1]:
# !pip install unidecode
# !pip install --upgrade tables

In [2]:
# Imports
import os; import sys;
import tqdm
import requests
import mimetypes
import zipfile
import math
import pandas as pd
pd.set_option('display.max_columns', None)
# LLpath = os.getcwd()+'/drive/MyDrive/Data201920/LL201920/'
LLpath = os.getcwd()+'\\LL201920\\'
BLpath = os.getcwd()+'\\BL201920\\'
L1path = os.getcwd()+'\\L1201920\\'
SApath = os.getcwd()+'\\SA201920\\'
import ws as ws

In [3]:
# Create spadl and raw data folders
spadl_datafolder = os.getcwd()+'\\spadl\\'
raw_datafolder = os.getcwd()+'\\raw\\'

# Create data folder if it doesn't exist
for d in [raw_datafolder, spadl_datafolder]:
    if not os.path.exists(d):
        os.makedirs(d, exist_ok=True)
        print(f"Directory {d} created ")

In [4]:
json_files = [LLpath+pos_json for pos_json in os.listdir(LLpath) if pos_json.endswith('.json')] + \
            [BLpath+pos_json for pos_json in os.listdir(BLpath) if pos_json.endswith('.json')] + \
            [L1path+pos_json for pos_json in os.listdir(L1path) if pos_json.endswith('.json')] + \
            [SApath+pos_json for pos_json in os.listdir(SApath) if pos_json.endswith('.json')]
w_h5 = os.path.join(raw_datafolder, 'w.h5')
ws.jsonfiles_to_h5(json_files,w_h5,append=True)

  0%|                                                                                         | 0/1344 [00:00<?, ?it/s]C:\Users\bosem\anaconda3\lib\site-packages\ipykernel_launcher.py:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['qualifiers'], dtype='object')]

  
100%|██████████████████████████████████████████████████████████████████████████████| 1344/1344 [03:09<00:00,  7.08it/s]
C:\Users\bosem\anaconda3\lib\site-packages\ipykernel_launcher.py:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['side', 'score'], dtype='object')]

  


In [5]:
spadl_h5 = os.path.join(spadl_datafolder, "spadl.h5")

In [6]:
ws.convert_to_spadl(w_h5,spadl_h5)

C:\Users\bosem\anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['side', 'score'], dtype='object')]

  """Entry point for launching an IPython kernel.
100%|██████████████████████████████████████████████████████████████████████████████| 1344/1344 [04:09<00:00,  5.38it/s]


In [7]:
# Read games
games = pd.read_hdf(spadl_h5,"games")
print("nb of games:", len(games))

nb of games: 1344


In [8]:
players = pd.read_hdf(spadl_h5,"players")
teams = pd.read_hdf(spadl_h5,"teams")
actiontypes = pd.read_hdf(spadl_h5, "actiontypes")
bodyparts = pd.read_hdf(spadl_h5, "bodyparts")
results = pd.read_hdf(spadl_h5, "results")
# players['player_id'] = players['player_id']

In [25]:
import atomic as atomicspadl
import config as spadl

In [26]:
atomicactions = {}
for game in tqdm.tqdm(list(games.itertuples())):
    actions = pd.read_hdf(spadl_h5,f"actions/game_{game.game_id}")
    actions['action_id'] = range(len(actions))
    atomicactions[game.game_id] = atomicspadl.convert_to_atomic(actions)

100%|██████████████████████████████████████████████████████████████████████████████| 1344/1344 [03:07<00:00,  7.18it/s]


In [29]:
atomic_spadl_h5 = os.path.join(spadl_datafolder, "atomic-spadl.h5")

# Store all spadl data in h5-file
with pd.HDFStore(atomic_spadl_h5) as spadlstore:
    spadlstore["games"] = games
    spadlstore["teams"] = teams
    spadlstore["players"] = players
    for game_id in atomicactions.keys():
        spadlstore[f"atomic_actions/game_{game_id}"] = atomicactions[game_id]

    spadlstore["results"] = spadl.results_df()
    spadlstore["bodyparts"] = spadl.bodyparts_df()
    spadlstore["atomic_actiontypes"] = atomicspadl.actiontypes_df()

In [30]:
# Read games back from atomic spadl to check
agames = pd.read_hdf(atomic_spadl_h5,"games")
print("nb of games:", len(games))

nb of games: 1344
